In [36]:
import numpy as np
import pandas as pd
import codecs

import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
#Todo: to lowercase out tokens
#Todo: replace original with humour word


In [130]:
# Load data
train_df = pd.read_csv('../task-1/train.csv')
test_df = pd.read_csv('../task-1/dev.csv')

training_data = train_df['original']
test_data = test_df['original']
training_data

0       France is ‘ hunting down its citizens who join...
1       Pentagon claims 2,000 % increase in Russian tr...
2       Iceland PM Calls Snap Vote as Pedophile Furor ...
3       In an apparent first , Iran and Israel <engage...
4       Trump was told weeks ago that Flynn misled <Vi...
                              ...                        
9647    State officials blast ' unprecedented ' DHS <m...
9648    Protesters Rally for <Refugees/> Detained at J...
9649    Cruise line Carnival Corp. joins the fight aga...
9650    Columbia police hunt woman seen with <gun/> ne...
9651    Here 's What 's In The House-Approved Health <...
Name: original, Length: 9652, dtype: object

In [49]:
train_df

,id,original1,edit1,grades1,meanGrade1,original2,edit2,grades2,meanGrade2,label
0,10920-9866,""" Gene Cernan , Last <Astronaut/> on the Moon ...",Dancer,1113,1.2,""" Gene Cernan , Last Astronaut on the Moon , <...",impregnated,30001,0.8,1
1,3176-10722,""" I 'm done "" : Fed up with California , some ...",vagrants,1200,0.6,""" I 'm done "" : Fed up with <California/> , so...",pancakes,10110,0.6,0
2,3176-3702,""" I 'm done "" : Fed up with California , some ...",vagrants,1200,0.6,""" I 'm done "" : Fed up with <California/> , so...",life,2,0.4,1
3,10722-3702,""" I 'm done "" : Fed up with <California/> , so...",pancakes,10110,0.6,""" I 'm done "" : Fed up with <California/> , so...",life,2,0.4,1
4,12282-2083,""" Our expectations of what civic engagement lo...",imagine,0,0.0,""" Our expectations of what civic engagement <l...",smells,100220010,0.6,2
...,...,...,...,...,...,...,...,...,...,...
9376,975-13357,"“ It ’s painfully obvious "" Mueller will charg...",battery,1,0.2,"“ It ’s painfully obvious "" Mueller will charg...",plumbing,11103,1.2,2
9377,975-11773,"“ It ’s painfully obvious "" Mueller will charg...",battery,1,0.2,"“ It ’s painfully obvious "" Mueller will <char...",strangle,22331,2.2,2
9378,13357-11773,"“ It ’s painfully obvious "" Mueller will charg...",plumbing,11103,1.2,"“ It ’s painfully obvious "" Mueller will <char...",strangle,22331,2.2,2
9379,14954-14479,"“ Kompromat , ” media ethics and the law : Wha...",porn,20101,0.8,"“ Kompromat , ” media ethics and the law : Wha...",dance,32112,1.8,2


In [8]:
# To create our vocab
def create_vocab(data):
    """
    Creating a corpus of all the tokens used
    """
    tokenized_corpus = [] # Let us put the tokenized corpus in a list

    for sentence in data:

        tokenized_sentence = []

        for token in sentence.split(' '): # simplest split is

            tokenized_sentence.append(token)

        tokenized_corpus.append(tokenized_sentence)

    # Create single list of all vocabulary
    vocabulary = []  # Let us put all the tokens (mostly words) appearing in the vocabulary in a list

    for sentence in tokenized_corpus:

        for token in sentence:

            if token not in vocabulary:

                if True:
                    vocabulary.append(token)

    return vocabulary, tokenized_corpus

In [38]:
training_vocab, training_tokenized_corpus = create_vocab(training_data)
test_vocab, test_tokenized_corpus = create_vocab(test_data)
joint_vocab, joint_tokenized_corpus = create_vocab(pd.concat([training_data, test_data]))


In [34]:
print(train_df['original1'].iloc[0])
print(training_vocab[:20])
print(training_tokenized_corpus[0])


" Gene Cernan , Last <Astronaut/> on the Moon , Dies at 82 "
['"', 'Gene', 'Cernan', ',', 'Last', '<Astronaut/>', 'on', 'the', 'Moon', 'Dies', 'at', '82', 'I', "'m", 'done', ':', 'Fed', 'up', 'with', 'California']
['"', 'Gene', 'Cernan', ',', 'Last', '<Astronaut/>', 'on', 'the', 'Moon', ',', 'Dies', 'at', '82', '"']


In [41]:
# We create representations for our tokens
wvecs = [] # word vectors
word2idx = [] # word2index
idx2word = []


# This is a large file, it will take a while to load in the memory!
with codecs.open('glove.6B.100d.txt', 'r','utf-8') as f:
  index = 1
  for line in f.readlines():
    # Ignore the first line - first line typically contains vocab, dimensionality
    if len(line.strip().split()) > 3:
      word = line.strip().split()[0]
      if word in joint_vocab:
          (word, vec) = (word,
                     list(map(float,line.strip().split()[1:])))
          wvecs.append(vec)
          word2idx.append((word, index))
          idx2word.append((index, word))
          index += 1

wvecs = np.array(wvecs)
word2idx = dict(word2idx)
idx2word = dict(idx2word)

In [48]:
word2idx['astronaut']

3688

In [68]:
#Spacy tokenizer
from spacy.tokenizer import Tokenizer
import re


replacement_re = re.compile(r'^<*/>') #do not split replacement format
prefix_re = re.compile(r'''^[\[\("]''')
suffix_re = re.compile(r''',[\]\)"']$''')
infix_re = re.compile(r'''[-\,.~]''')


def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab,
                     token_match = replacement_re.match,
                     prefix_search=prefix_re.search,
                     suffix_search=suffix_re.search,
                     infix_finditer = infix_re.finditer
                    
                    
                    )
nlp = spacy.load("en_core_web_sm")
nlp.tokenizer = custom_tokenizer(nlp)

In [69]:
doc = nlp(train_df['original1'].iloc[6])
for token in doc:
    print(token.text)

"
System
safeguards
are
lacking
"
,
quote
following
a
Tesla
's
<crash/>
during
autopilot


In [79]:
training_data

0       " Gene Cernan , Last <Astronaut/> on the Moon ...
1       " I 'm done " : Fed up with California , some ...
2       " I 'm done " : Fed up with California , some ...
3       " I 'm done " : Fed up with <California/> , so...
4       " Our expectations of what civic engagement lo...
                              ...                        
9376    “ It ’s painfully obvious " Mueller will charg...
9377    “ It ’s painfully obvious " Mueller will charg...
9378    “ It ’s painfully obvious " Mueller will charg...
9379    “ Kompromat , ” media ethics and the law : Wha...
9380    “ Son of a bitch ” : Trump ’s NFL <outburst/> ...
Name: original1, Length: 9381, dtype: object

In [126]:
# To create our vocab

punctuation = "\":\.,"

def preprocessor(data,edits):
    """
    Creating a corpus of all the tokens used
    """
    tokenized_corpus_1= [] # Let us put the tokenized corpus in a list
    tokenized_corpus_2= []
    
    
    for i, sentence in enumerate(data):

        sentence = sentence.lower()
        tokenized_sentence_1 = []
        tokenized_sentence_2 = []

        for token in nlp(sentence): # simplest split is

            if token.text in punctuation:
                continue
            else:
                
                if token.text[0] == '<':
                    tokenized_sentence_1.append("???")
                    tokenized_sentence_2.append("???")
                    
                    tokenized_sentence_2.append(edits[i])
                    
                    tokenized_sentence_1.append(token.text[1:-2])
                else:
                    tokenized_sentence_2.append(token.text)
                
                    tokenized_sentence_1.append(token.text)

        tokenized_corpus_1.append(tokenized_sentence_1)
        tokenized_corpus_2.append(tokenized_sentence_2)

    # Create single list of all vocabulary
    vocabulary = []  # Let us put all the tokens (mostly words) appearing in the vocabulary in a list

    for sentence in tokenized_corpus_1:

        for token in sentence:

            if token not in vocabulary:

                if True:
                    vocabulary.append(token)
    
    for token in edits:
        
        if token not in vocabulary:
            
            vocabulary.append(token)

    return vocabulary, tokenized_corpus_1, tokenized_corpus_2

In [127]:
training_vocab, training_tokenized_corpus_1,training_tokenized_corpus_2 = preprocessor(training_data[:10],train_df['edit1'][:10])

In [128]:
print(training_tokenized_corpus_1[1])
print(training_tokenized_corpus_2[1])

['i', "'m", 'done', 'fed', 'up', 'with', 'california', 'some', '???', 'conservatives', 'look', 'to', 'texas']
['i', "'m", 'done', 'fed', 'up', 'with', 'california', 'some', '???', 'vagrants', 'look', 'to', 'texas']


In [95]:
train_df['edit1']

0         Dancer
1       vagrants
2       vagrants
3       pancakes
4        imagine
          ...   
9376     battery
9377     battery
9378    plumbing
9379        porn
9380     tantrum
Name: edit1, Length: 9381, dtype: object

In [94]:
training_tokenized_corpus

[[gene, cernan, last, <astronaut/>, on, the, moon, dies, at, 82],
 [i,
  'm,
  done,
  fed,
  up,
  with,
  california,
  some,
  <conservatives/>,
  look,
  to,
  texas],
 [i,
  'm,
  done,
  fed,
  up,
  with,
  california,
  some,
  <conservatives/>,
  look,
  to,
  texas],
 [i,
  'm,
  done,
  fed,
  up,
  with,
  <california/>,
  some,
  conservatives,
  look,
  to,
  texas],
 [our,
  expectations,
  of,
  what,
  civic,
  engagement,
  looks,
  like,
  do,
  n’t,
  match,
  reality,
  can,
  we,
  <fix/>,
  that,
  ?],
 [system,
  safeguards,
  are,
  lacking,
  quote,
  following,
  a,
  tesla,
  's,
  <crash/>,
  during,
  autopilot],
 [system,
  safeguards,
  are,
  lacking,
  quote,
  following,
  a,
  tesla,
  's,
  <crash/>,
  during,
  autopilot],
 [system,
  safeguards,
  are,
  lacking,
  quote,
  following,
  a,
  tesla,
  's,
  <crash/>,
  during,
  autopilot],
 [we,
  could,
  be,
  separated,
  immigrants,
  families,
  react,
  after,
  trump,
  <administration/>,
 